In [67]:
%pip install gradio joblib numpy

Note: you may need to restart the kernel to use updated packages.


In [68]:
import gradio as gr
import joblib
import numpy as np

In [69]:
# Memuat model yang telah disimpan
model_path = "../bin/model.pkl"
model = joblib.load(model_path)


# Fungsi untuk melakukan prediksi
def predict_stroke(
    gender,
    age,
    hypertension,
    heart_disease,
    ever_married,
    work_type,
    residence_type,
    avg_glucose_level,
    bmi_choice,
    bmi,
    weight,
    height,
    smoking_status,
):
    # Melakukan encoding input yang sesuai dengan proses sebelumnya
    label_encoders = {
        "gender": {"Male": 1, "Female": 0, "Other": 2},
        "ever_married": {"Yes": 1, "No": 0},
        "work_type": {
            "Private": 2,
            "Self-employed": 3,
            "Govt_job": 0,
            "children": 4,
            "Never_worked": 1,
        },
        "residence_type": {"Urban": 1, "Rural": 0},
        "smoking_status": {
            "formerly smoked": 1,
            "never smoked": 2,
            "smokes": 3,
            "Unknown": 0,
        },
        "hypertension": {"Yes": 1, "No": 0},
        "heart_disease": {"Yes": 1, "No": 0},
    }

    # Encode categorical inputs
    gender = label_encoders["gender"].get(gender, -1)
    ever_married = label_encoders["ever_married"].get(ever_married, -1)
    work_type = label_encoders["work_type"].get(work_type, -1)
    residence_type = label_encoders["residence_type"].get(residence_type, -1)
    smoking_status = label_encoders["smoking_status"].get(smoking_status, -1)
    hypertension = label_encoders["hypertension"].get(hypertension, -1)
    heart_disease = label_encoders["heart_disease"].get(heart_disease, -1)

    # Pastikan semua data tersedia
    if -1 in [
        gender,
        ever_married,
        work_type,
        residence_type,
        smoking_status,
        hypertension,
        heart_disease,
    ]:
        return (
            "Error: Invalid input detected. Please ensure all inputs are correct.",
            None,
            None,
            None,
        )

    # Calculate BMI if needed
    if bmi_choice == "Automatic":
        if weight <= 0 or height <= 0:
            return "Error: Invalid weight or height for BMI calculation.", None, None, None
        bmi = weight / ((height / 100) ** 2)

    # Buat array fitur input
    input_data = np.array(
        [
            [
                gender,
                age,
                hypertension,
                heart_disease,
                ever_married,
                work_type,
                residence_type,
                avg_glucose_level,
                bmi,
                smoking_status,
            ]
        ]
    )

    # Melakukan prediksi
    prediction = model.predict(input_data)
    probability = model.predict_proba(input_data)[0][1]  # Probabilitas kelas positif

    # Output hasil prediksi
    if prediction[0] == 1:
        prediction_result = "High Risk"
    else:
        prediction_result = "Low Risk"

    # Menyusun penjelasan fitur berdasarkan tingkatan
    explanation = []

    # Penjelasan untuk age
    if age < 40:
        explanation.append("<li>Your age is in the low-risk range for stroke.</li>")
    elif 40 <= age < 60:
        explanation.append(
            "<li>Your age is in the moderate-risk range for stroke.</li>"
        )
    else:
        explanation.append("<li>Your age is in the high-risk range for stroke.</li>")

    # Penjelasan untuk BMI
    if bmi < 18.5:
        explanation.append(
            "<li>Your BMI indicates that you are underweight. This can increase health risks.</li>"
        )
    elif 18.5 <= bmi < 24.9:
        explanation.append(
            "<li>Your BMI is in the normal range, which is generally healthy.</li>"
        )
    elif 25 <= bmi < 29.9:
        explanation.append(
            "<li>Your BMI indicates that you are overweight, which can increase health risks.</li>"
        )
    else:
        explanation.append(
            "<li>Your BMI indicates that you are obese, which significantly increases health risks.</li>"
        )

    # Penjelasan untuk Average Glucose Level
    if avg_glucose_level < 70:
        explanation.append(
            "<li>Your average glucose level is low, which is generally safe.</li>"
        )
    elif 70 <= avg_glucose_level < 100:
        explanation.append("<li>Your average glucose level is normal.</li>")
    else:
        explanation.append(
            "<li>Your average glucose level is high, which can increase your risk of stroke.</li>"
        )

    # Mengembalikan empat output: hasil prediksi, probabilitas, hasil BMI, dan penjelasan fitur dalam format HTML
    return (
        prediction_result,
        f"{probability * 100:.2f}%",
        (f"BMI: {bmi:.2f}" if bmi_choice == "Automatic" else None),
        f"<ul>{''.join(explanation)}</ul>",
    )


# Desain interface Gradio
with gr.Blocks() as app:
    gr.Markdown("## Stroke Prediction Application")

    with gr.Row():
        with gr.Column():
            gender = gr.Dropdown(
                label="Gender", choices=["Male", "Female", "Other"], value="Male"
            )
            age = gr.Number(label="Age", value=45)
            hypertension = gr.Radio(label="Hypertension", choices=["Yes", "No"], value="No")
            heart_disease = gr.Radio(
                label="Heart Disease", choices=["Yes", "No"], value="No"
            )
            ever_married = gr.Radio(label="Ever Married", choices=["Yes", "No"], value="No")
            work_type = gr.Dropdown(
                label="Work Type",
                choices=[
                    "Private",
                    "Self-employed",
                    "Govt_job",
                    "children",
                    "Never_worked",
                ],
                value="Private",
            )
            residence_type = gr.Radio(
                label="Residence Type", choices=["Urban", "Rural"], value="Urban"
            )
            avg_glucose_level = gr.Number(label="Average Glucose Level", value=85.0)
            bmi_choice = gr.Radio(
                label="BMI Input Method", choices=["Manual", "Automatic"], value="Manual"
            )
            bmi = gr.Number(label="BMI", value=22.5, visible=True)
            weight = gr.Number(label="Weight (kg)", value=70, visible=False)
            height = gr.Number(label="Height (cm)", value=170, visible=False)
            smoking_status = gr.Dropdown(
                label="Smoking Status",
                choices=["formerly smoked", "never smoked", "smokes", "Unknown"],
                value="never smoked",
            )
            submit = gr.Button("Predict")

        with gr.Column():
            output_prediction = gr.Textbox(label="Prediction Result")
            output_probability = gr.Textbox(label="Prediction Probability")
            output_bmi = gr.Textbox(label="Calculated BMI", visible=False)
            output_explanation = gr.HTML(label="Feature Explanation")

    def update_bmi_visibility(bmi_choice):
        if bmi_choice == "Manual":
            return (
                gr.update(visible=True),
                gr.update(visible=False),
                gr.update(visible=False),
                gr.update(visible=False),
            )
        else:
            return (
                gr.update(visible=False),
                gr.update(visible=True),
                gr.update(visible=True),
                gr.update(visible=True),
            )

    bmi_choice.change(
        update_bmi_visibility, inputs=[bmi_choice], outputs=[bmi, weight, height, output_bmi]
    )

    submit.click(
        predict_stroke,
        inputs=[
            gender,
            age,
            hypertension,
            heart_disease,
            ever_married,
            work_type,
            residence_type,
            avg_glucose_level,
            bmi_choice,
            bmi,
            weight,
            height,
            smoking_status,
        ],
        outputs=[output_prediction, output_probability, output_bmi, output_explanation],
    )


In [70]:
app.launch()

* Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.
